# Import packages

In [1]:
import pandas as pd
import sys
sys.path.append('../')

from aqc_utils.molecule import molecule
from aqc_utils.slurm_manager import slurm_manager
from aqc_utils.helper_classes import slurm_status

import logging
logging.basicConfig(level=logging.INFO)

# Initialize the slurm manager

Slurm manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [2]:
sm=slurm_manager(user='jschleinitz', host='hermite.chimie.ens.fr')
sm.connect()

INFO:aqc_utils.slurm_manager:Creating connection to hermite.chimie.ens.fr as jschleinitz
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:aqc_utils.slurm_manager:Connected to hermite.chimie.ens.fr as jschleinitz is True.


# Provide smiles string(s)
and copy into them into my_smiles variable. It's a list of strings (a Python list can be of any size), the smiles are extracted for the csv resulting form the step 2 numbering the molecules.

In [3]:
react_nb = pd.read_csv("../data_csv/Data_test09032021.csv", sep=',')
import numpy as np
from rdkit import Chem

In [4]:
my_smiles = list(react_nb["Base/additif après correction effective"])
unik_smi = []
for smi in my_smiles:
    try:
        smi_can = Chem.MolToSmiles(Chem.MolFromSmiles(smi))
        L = smi_can.split('.')
        for sub_smi in L:
            sub_smi_can = Chem.MolToSmiles(Chem.MolFromSmiles(sub_smi))
            if sub_smi_can not in unik_smi:
                unik_smi.append(sub_smi_can)
    except:
        pass
unik_smi

RDKit ERROR: [19:24:21] SMILES Parse Error: syntax error while parsing: c1ccccc1[Mg]Br.[Sc+++].CC(C)(C)S(=O)(=O)[O-].CC(C)(C)S(=O)(=O)[O-].CC(C)(C)S(=O)(=O)[O-]
[19:24:21] SMILES Parse Error: syntax error while parsing: c1ccccc1[Mg]Br.[Sc+++].CC(C)(C)S(=O)(=O)[O-].CC(C)(C)S(=O)(=O)[O-].CC(C)(C)S(=O)(=O)[O-]
RDKit ERROR: [19:24:21] SMILES Parse Error: Failed parsing SMILES 'c1ccccc1[Mg]Br.[Sc+++].CC(C)(C)S(=O)(=O)[O-].CC(C)(C)S(=O)(=O)[O-].CC(C)(C)S(=O)(=O)[O-]' for input: 'c1ccccc1[Mg]Br.[Sc+++].CC(C)(C)S(=O)(=O)[O-].CC(C)(C)S(=O)(=O)[O-].CC(C)(C)S(=O)(=O)[O-]'
RDKit ERROR: [19:24:21] SMILES Parse Error: syntax error while parsing: c1ccccc1[Mg]Br.[Ti++++].CC(C)[O-].CC(C)[O-].CC(C)[O-].CC(C)[O-]
RDKit ERROR: [19:24:21] SMILES Parse Error: Failed parsing SMILES 'c1ccccc1[Mg]Br.[Ti++++].CC(C)[O-].CC(C)[O-].CC(C)[O-].CC(C)[O-]' for input: 'c1ccccc1[Mg]Br.[Ti++++].CC(C)[O-].CC(C)[O-].CC(C)[O-].CC(C)[O-]'
RDKit ERROR: [19:24:21] SMILES Parse Error: syntax error while parsing: c1ccccc1[Mg]Br.

['[Li]C',
 'O=C([O-])[O-]',
 '[K+]',
 '[Cs+]',
 'CC(C)(C)[O-]',
 'O',
 'O=P([O-])([O-])[O-]',
 'Cc1ccccc1[Mg]Br',
 '[Cl-]',
 '[Li+]',
 'Br[Mg]c1ccccc1',
 '[Mg+2]',
 'C[Mg]Br',
 'CC[Mg]Br',
 'Br[Mg]C1CCCCC1',
 'Br[Mg]Cc1ccccc1',
 'Cc1ccc([Mg]Br)cc1',
 'C[Mg]I',
 'CC1(C)CCCC(C)(C)N1[O-]',
 'C=C(c1ccccc1)c1ccccc1',
 'Cc1cccc(C)c1[Mg]Br',
 'Cc1cc(C)c([Mg]Br)c(C)c1',
 'Cl[Mg]c1ccccc1',
 '[O-]',
 '[Na+]',
 '[F-]',
 '[Mn]',
 '[Zn]',
 'O=P([O-])(O)O',
 '[I-]',
 'C[Si](C)(C)Cl',
 '[Zn+2]',
 'CC[N+](CC)(CC)CC',
 'Oc1ccccc1',
 'CCN(CC)CC',
 'CC(C)(C)C(=O)O',
 '[O-]C([O-])O',
 '[Rb+]',
 'CC(C)(C)C(=O)[O-]',
 '[Ag+]',
 '[Cu+2]',
 'CCCC[N+](CCCC)(CCCC)CCCC',
 '[Br-]',
 'O=S(=O)([O-])[O-]',
 '[Cu+]',
 'Cc1ccccc1',
 'O=S(=O)([O-])C(F)(F)F',
 '[Al]',
 'O=P([O-])([O-])O',
 'CC(=O)[O-]',
 'C1CCC2=NCCCN2CC1',
 'C[Al](C)C',
 '[Sr+2]',
 'CCC[N+](CCC)(CCC)CCC',
 'C[N+](C)(C)C',
 'CCOCC',
 'C1=CCCC=CCC1',
 'CC[O-]',
 'CC1(C)OB(c2ccccc2)OC1(C)C',
 '[O-]P([O-])[O-]',
 'CC(C)(C)O[Al-](OC(C)(C)C)OC(C)(C)C',
 'Cl[

# Initialize the molecules and generate conformations
We will use some for loops to loop over the smiles, only one conformation will be taken for each molecules in this work

In [5]:
mols = []
for smile in unik_smi:
    mols.append(molecule(smile, max_num_conformers=1))

INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: [Li]C
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: [O-]C(=O)[O-]
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: [K+]
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: [Cs+]
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: [O-]C(C)(C)C
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_

INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: [Ag+]
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: [Cu+2]
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: CCCC[N+](CCCC)(CCCC)CCCC
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: [Br-]
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: [O-]S(=O)(=O)[O-

###### Draw some molecules

In [6]:
#mols[0].draw(2)

# Create gaussian job for each conformation

In [7]:
for mol in mols:
    sm.create_jobs_for_molecule(mol, workflow_type="equilibrium")

INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


CO3--_f128_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


K+_6b55_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


...

Cl4Zr_7d7d_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


ClLi_e823_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C12H27AlO3_e518_conf_0


In [8]:
sm.get_job_stats(split_by_can=True)

status,created,done,failed,incomplete,uploaded
can,,,,,
BrB(Br)Br,1,0,0,0,0
Br[Mg]C1CCCCC1,1,0,0,0,0
Br[Mg]Cc1ccccc1,1,0,0,0,0
Br[Mg]c1c(C)cc(cc1C)C,1,0,0,0,0
C(Oc1ccccc1)Oc1ccccc1,0,0,1,0,0
...,...,...,...,...,...
[Rb+],1,0,0,0,0
[Sr+2],1,0,0,0,0
[Y+3],1,0,0,0,0


In [ ]:
sm.submit_jobs()

INFO:aqc_utils.slurm_manager:Submitting 28 jobs.
INFO:aqc_utils.slurm_manager:Connection got disconnected, reconnecting.
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:aqc_utils.slurm_manager:Connected to hermite.chimie.ens.fr as jschleinitz is True.
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)


Submitted batch job 10321


INFO:aqc_utils.slurm_manager:Submitted job f782e3995e5e24e0f7147818fd7dd985, job_id: 10321.


Submitted batch job 10322


INFO:aqc_utils.slurm_manager:Submitted job 85ecba28653268dda0c5050eb383fce3, job_id: 10322.


...

INFO:aqc_utils.slurm_manager:Submitted job ef9a7ab14d48dce5f59b7dc66134faaa, job_id: 10344.


Submitted batch job 10345


INFO:aqc_utils.slurm_manager:Submitted job a98da6d62b840a16bc5c9bc968fb27c9, job_id: 10345.


Submitted batch job 10346


INFO:aqc_utils.slurm_manager:Submitted job 3d8ff2fd6b5e265f12e92b523205f833, job_id: 10346.


In [ ]:
sm.squeue()

In [90]:
for job in sm.get_jobs():
    if str(sm.get_jobs()[job].status) == 'slurm_status.done':
        print(sm.get_jobs()[job].base_name)

C14H18OSi_95fa_conf_0
C11H10O2_d7c7_conf_0
C11H10O_9eb4_conf_0
C11H17NO_9080_conf_0
C11H17NO_d167_conf_0
C10H11NO_10ad_conf_0
C12H12O_d53c_conf_0
C13H14O_eb43_conf_0


In [84]:
job

'6625625ec8945f514e3c68c6d47152bb'

In [85]:
job_all = sm.get_jobs()[job]

In [33]:
sm.get_job_stats()

status,done,failed,incomplete,uploaded
jobs,1,338,1,19


In [87]:
str(job_all.status)

'slurm_status.created'

In [88]:
job_all.base_name

'C12H13NO2_b9f2_conf_0'